In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
import time
import random
import requests
import os
import glob
import re
import csv


class MOST_whole_year():

    def __init__(self, driver_path, download_path,
                 crawler_year):  #Class member
        self.url_to_crawl = 'https://wsts.nstc.gov.tw/STSWeb/Award/AwardMultiQuery.aspx'
        self.driver_path = driver_path
        self.crawler_year = crawler_year
        self.SaveFile_Path = SaveFile_Path
        self.SaveFile_Name = SaveFile_Name        
        
    def start(self):
        self.browser = webdriver.Chrome(self.driver_path)
        self.browser.maximize_window()

    def get_page(self, url):
        self.browser.get(url)
        time.sleep(3)

    def search_work(self):
        #點擊「科技部補助研究計畫」
        self.browser.find_element(By.ID, 'dtlItem_btnItem_0').click()
        Select(self.browser.find_element(By.XPATH, '//*[@id="wUctlAwardQueryPage_repQuery_ddlYRst_0"]')).select_by_value(str(crawler_year))
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"  可視為「等待」的指令
            EC.presence_of_element_located((By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
        Select(self.browser.find_element(By.XPATH, '//*[@id="wUctlAwardQueryPage_repQuery_ddlYRend_0"]')).select_by_value(str(crawler_year))
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"  可視為「等待」的指令
            EC.presence_of_element_located((By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
        # 設定顯示頁數  選到每頁200筆資料 
        self.browser.find_element(By.ID, 'wUctlAwardQueryPage_ddlPageSize').click()
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"  可視為「等待」的指令
            EC.presence_of_element_located((By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
        self.browser.find_element(By.XPATH,'//*[@id="wUctlAwardQueryPage_ddlPageSize"]/option[6]').click()
        WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"  可視為「等待」的指令
            EC.presence_of_element_located((By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
        #解析html架構並儲存成資料
        soup = BeautifulSoup(self.browser.page_source, 'lxml')
        # print(soup.prettify())
        print('搜尋時間： ' + time.strftime("%y/%m/%d") + ' ' +time.strftime("%H:%M:%S"))
        # 確認頁數
        #pagevalue 是每一頁的顯示筆數，固定不動
        self.pagevalue = 200
        page_find_all = soup.find("span", id='wUctlAwardQueryPage_grdResult_lblPage').string
        page_str = ([int(page) for page in re.findall(r'-?\d+\.?\d*', page_find_all)])
        page_int = int(page_str[1])  #筆數
        page_last = page_int % int(self.pagevalue)
        if page_int % int(self.pagevalue) == 0:
            self.page_all = page_int // int(self.pagevalue)
        else:
            self.page_all = (page_int // int(self.pagevalue)) + 1  #頁數
        print('共 ' + str(page_int) + ' 筆')
        print('有 ' + str(self.page_all) + ' 頁')
        print('最後一頁剩 ' + str(page_last) + ' 筆')

    def download_file(self, soup):
        result_list = []
        try:
            for x in range(0, self.pagevalue):
                try:
                    self.title = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblAWARD_PLAN_CHI_DESCc_{}'.format(x)}).text
                except:
                    self.title = ''
                try:
                    self.starend = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblAWARD_ST_ENDc_{}'.format(x)}).text
                except:
                    self.starend = ''
                try:
                    self.cost = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblAWARD_TOT_AUD_AMTc_{}'.format(x)}).text
                except:
                    self.cost = ''
                try:
                    self.chkey = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblKEYS_CHIc_{}'.format(x)}).text
                except:
                    self.chkey = ''
                try:
                    self.engkey = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblKEYS_ENGc_{}'.format(x)}).text
                except:
                    self.engkey = ''
                try:
                    self.chabst = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblBRIEF_CHI_Sc_{}'.format(x)}).text
                except:
                    self.chabst = ''
                try:
                    self.engabst = soup.find("span", {'id':'wUctlAwardQueryPage_grdResult_lblBRIEF_ENG_Sc_{}'.format(x)}).text
                except:
                    self.engabst = ''
                result_list.append([self.title, self.starend, self.cost, self.chkey, self.engkey, self.chabst, self.engabst])
        except:
            print("爬取資料失敗")
        return result_list
        
    def roll_page(self):  #翻頁並進入download_file的步驟
        page = 1
        i = 1
        res_process = []
        res_all = pd.DataFrame()
        while page < self.page_all:
            soup = BeautifulSoup(self.browser.page_source, 'lxml')
            result_list = self.download_file(soup)
            time.sleep(5)
            result_df = pd.DataFrame(result_list)
            res_process = pd.DataFrame(res_process)
            res_process = res_process.append(result_df)
            
            page = page + 1
            i = i + 1
            self.browser.find_element(By.ID, "wUctlAwardQueryPage_grdResult_btnNext").click()  #點選下一頁
            WebDriverWait(self.browser, 30).until(  #等待網頁讀取 直到出現 "搜尋框"  可視為「等待」的指令
                EC.presence_of_element_located((By.ID, "wUctlAwardQueryPage_repQuery_txtT_4")))
    
            if page == self.page_all:
                print("此步驟為爬取最終頁資料")
                soup = BeautifulSoup(self.browser.page_source, 'lxml')
                result_list = self.download_file(soup)
                time.sleep(5)
                result_df = pd.DataFrame(result_list)
                res_process = pd.DataFrame(res_process)
                res_process = res_process.append(result_df)
                res_all = res_all.append(res_process)
                res_all.columns = ['計畫名稱', '研究期間', '研究經費', '中文關鍵字', '英文關鍵字', '中文摘要', '英文摘要']
                try:
                    res_all.to_csv(SaveFile_Path + '\\' + SaveFile_Name + "_" + "_" +time.strftime("%m%d") + "_" + time.strftime("%H%M") +".csv",
                    encoding="utf_8_sig",index=False)  #儲存結果
                    print("存檔成功")  #最終檔名會加上 1.月份 2.日期 3.時間(時分)
                    break
                except:
                    print("存檔失敗，請檢察結果檔是否正在開啟，或處於無法被寫入的狀態。")
                
    def close_driver(self):
        self.browser.quit()
        print('成功爬取MOST{}年度'.format(self.crawler_year))
        
    def step(self):
        self.start()
        self.get_page(self.url_to_crawl)
        print('正在爬取MOST{}年度'.format(self.crawler_year))
        self.search_work()
        self.roll_page()
        self.close_driver()

####################################################################################
if __name__ == '__main__':
    driver_path = r'C:\Users\user\Desktop\py\chromedriver.exe'  #chromedriver位置
    SaveFile_Path = r'C:\Users\user\Desktop\py\TEST'            #檔案下載位置
    SaveFile_Name = r'MOST_ALL'                                 #存檔檔名
    crawler_year = '110'                                        #爬取年度

    go = MOST_whole_year(driver_path, SaveFile_Path, crawler_year)
    go.step()

C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(self.driver_path)


正在爬取MOST110年度
搜尋時間： 22/08/03 16:29:02
共 13634 筆
有 69 頁
最後一頁剩 34 筆
此步驟為爬取第1頁資料


C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)


此步驟為爬取第2頁資料
此步驟為爬取第3頁資料
此步驟為爬取第4頁資料


C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)


...

此步驟為爬取第68頁資料


C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:122: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)


此步驟為爬取最終頁資料


C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_process = res_process.append(result_df)
C:\Users\user\AppData\Local\Temp\ipykernel_46476\3288874620.py:138: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_all = res_all.append(res_process)


爬蟲成功
成功爬取MOST110年度


NameError: name 'result_list' is not defined